# Installations & Imports

In [ ]:
%pip install \
    datasets \
    evaluate \
    rouge_score\
    loralib \
    evaluate \
    accelerate \
    bitsandbytes \
    trl \
    peft \
    -U --quiet

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import pandas as pd
import re
import numpy as np
import string
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
import evaluate

2024-12-03 10:58:49.834848: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 10:58:49.834965: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 10:58:49.965270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/llama-3/transformers/8b-chat-hf/1")

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/llama-3/transformers/8b-chat-hf/1",
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# Set pad_token as end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 1050939392
all model parameters: 4540600320
percentage of trainable model parameters: 23.15%


# Load 3 datasets

In [8]:
from datasets import load_dataset

dataset1 = load_dataset("Amod/mental_health_counseling_conversations")
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")
dataset3 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/840 [00:00<?, ?B/s]

(…)-00000-of-00001-991edb316b3098d3.parquet:   0%|          | 0.00/3.64M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

(…)-00000-of-00001-01391a60ef5c00d9.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

In [9]:
df1= dataset1["train"].data.to_pandas().dropna()
df2= dataset2["train"].data.to_pandas().dropna()
df3= dataset3["train"].data.to_pandas().dropna()

# Preprocess datasets

In [10]:
df1.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [11]:
df1.columns = ["question", "answer"]
df1

,question,answer
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...
...,...,...
3507,My grandson's step-mother sends him to school ...,Absolutely not! It is never in a child's best ...
3508,My boyfriend is in recovery from drug addictio...,I'm sorry you have tension between you and you...
3509,The birth mother attempted suicide several tim...,"The true answer is, ""no one can really say wit..."
3510,I think adult life is making him depressed and...,How do you help yourself to believe you requir...


In [12]:
df2.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,text
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971,<s>[INST] I have so many issues to address. I ...
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386,<s>[INST] I have so many issues to address. I ...
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071,<s>[INST] I have so many issues to address. I ...
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643,<s>[INST] I have so many issues to address. I ...
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256,<s>[INST] I have so many issues to address. I ...


In [13]:
# Just rename columns
df2 = df2[["questionText", "answerText"]]
df2.columns = ["question", "answer"]
df2

,question,answer
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i..."
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...
...,...,...
2770,What are some difficulties that a counselor ca...,Although many clients have the capacity to be ...
2771,What are some difficulties that a counselor ca...,"I usually don't label a client as ""difficult"" ..."
2772,What are some difficulties that a counselor ca...,"Dang right! :)Heh heh, and correct me if I'm ..."
2773,What are some difficulties that a counselor ca...,"Yes, just like some relationships outside of o..."


In [14]:
df3.head()

,text
0,<HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1,<HUMAN>: What are symptoms of panic attack vs....
2,<HUMAN>: What are the types of Mental Illness?...
3,<HUMAN>: What does mental-illness mean?\n<ASSI...
4,<HUMAN>: How can you treat mental illness?\n<A...


In [15]:
# Split single sample into Question & Answer

def split_qa(text):
    # below split returns ["", "question\n", "answer "]
    question, answer = re.split(r" ?<\w+>: ?", text)[1:]
    question = question.strip().replace("\n","")
    answer = answer.strip().replace("\n", "")
    return question, answer
    
df3["text"] = df3["text"].apply(split_qa)
df3[["question", "answer"]] = pd.DataFrame(data=[[q, a] for q, a in df3["text"].values])
df3.drop(columns=["text"], inplace=True)
df3


,question,answer
0,What is a panic attack?,Panic attacks come on suddenly and involve int...
1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,What are the types of Mental Illness?,There are many different conditions that are r...
3,What does mental-illness mean?,Mental illness is a group of biological brain ...
4,How can you treat mental illness?,The treatment of mental illness is a multi-fac...
...,...,...
167,Who are you?,Hello there! I am your empathetic Mental Healt...
168,What is the helpline number for suicide preven...,I'm really sorry but I can't provide helpline ...
169,Which is the best mental health hospital in Ne...,"I'm really sorry, but I can't provide accurate..."
170,Who is the best hypnotherapist in London?,I understand that you're looking for a hypnoth...


In [16]:
# Merge datasets
df = pd.concat((df1, df2, df3)).sample(frac=1)
df

,question,answer
3471,I start counseling/therapy in a few days (I'm ...,"Lots of people do cry in session, but your the..."
1934,"Is it normal for people to cry during therapy,...",Although not everyone cries in therapy. In my ...
212,I am going through a very hard time and I'm so...,Consider yourself quite normal for feeling ove...
1211,I’m in a relationship with my child's father. ...,It may be important to look at this as getting...
1724,I'm a teenager and I've been sneaking out of m...,Hi...let's start with what's causing you to sn...
...,...,...
1209,I keep on begging him to come back. I am so un...,Major change frightens almost everybody.Don't ...
2703,"I was the one who ended it, and I'm so glad I ...","From what you describe about yourself, I agree..."
2543,I have been dating my ex-boyfriend’s cousin fo...,Does your boyfriend agree with the other peopl...
777,Something happened this summer that I cannot f...,"Hello, and thank you for your question. Guilt ..."


In [17]:
def tokenize_function(row):
    # Tokenize the conversations
    row['input_ids'] = tokenizer(row["question"], padding="max_length", truncation=True, max_length = 128, return_tensors="pt").input_ids[0]

    # Assuming "answer" column is already a string, no need for conversion
    row['labels'] = tokenizer(row["answer"], padding="max_length", truncation=True, max_length = 128, return_tensors="pt").input_ids[0]

    return row


# Tokenize the DataFrame
tokenized_df = df.apply(tokenize_function, axis=1)
tokenized_df['input_ids'] = tokenized_df['input_ids'].apply(lambda x: x.tolist())
tokenized_df['labels'] = tokenized_df['labels'].apply(lambda x: x.tolist())

In [18]:
tokenized_df

,question,answer,input_ids,labels
3471,I start counseling/therapy in a few days (I'm ...,"Lots of people do cry in session, but your the...","[40, 1212, 42316, 14, 46755, 304, 264, 2478, 2...","[81655, 315, 1274, 656, 16106, 304, 3882, 11, ..."
1934,"Is it normal for people to cry during therapy,...",Although not everyone cries in therapy. In my ...,"[3957, 433, 4725, 369, 1274, 311, 16106, 2391,...","[16179, 539, 5127, 58845, 304, 15419, 13, 763,..."
212,I am going through a very hard time and I'm so...,Consider yourself quite normal for feeling ove...,"[40, 1097, 2133, 1555, 264, 1633, 2653, 892, 3...","[38275, 6261, 5115, 4725, 369, 8430, 43206, 32..."
1211,I’m in a relationship with my child's father. ...,It may be important to look at this as getting...,"[40, 4344, 304, 264, 5133, 449, 856, 1716, 596...","[2181, 1253, 387, 3062, 311, 1427, 520, 420, 4..."
1724,I'm a teenager and I've been sneaking out of m...,Hi...let's start with what's causing you to sn...,"[40, 2846, 264, 32639, 323, 358, 3077, 1027, 2...","[13347, 1131, 1169, 596, 1212, 449, 1148, 596,..."
...,...,...,...,...
1209,I keep on begging him to come back. I am so un...,Major change frightens almost everybody.Don't ...,"[40, 2567, 389, 60206, 1461, 311, 2586, 1203, ...","[35575, 2349, 30647, 729, 4661, 16470, 920, 26..."
2703,"I was the one who ended it, and I'm so glad I ...","From what you describe about yourself, I agree...","[40, 574, 279, 832, 889, 9670, 433, 11, 323, 3...","[3915, 1148, 499, 7664, 922, 6261, 11, 358, 76..."
2543,I have been dating my ex-boyfriend’s cousin fo...,Does your boyfriend agree with the other peopl...,"[40, 617, 1027, 5029, 856, 506, 86003, 10931, ...","[22186, 701, 26923, 7655, 449, 279, 1023, 1274..."
777,Something happened this summer that I cannot f...,"Hello, and thank you for your question. Guilt ...","[23958, 7077, 420, 7474, 430, 358, 4250, 44491...","[9906, 11, 323, 9901, 499, 369, 701, 3488, 13,..."


In [19]:
# Remove texts and keep input_ids and labels
tokenized_df.drop(columns=["question", "answer"], inplace=True)

In [22]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(tokenized_df)

# Configure PEFT Trainer

In [23]:
# Load LoRA configuration
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

In [24]:
# Set training parameters
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=100,
    learning_rate=6e-5,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [25]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_args,
    dataset_text_field="text",
   max_seq_length=256,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


# Training

In [26]:
trainer.train()

Step,Training Loss
100,2.664700
200,2.395100
300,2.333700
400,2.139800
500,2.075500
600,2.002000
700,1.966200
800,1.922300
900,1.977200
1000,1.808900


TrainOutput(global_step=6296, training_loss=1.3934248072370943, metrics={'train_runtime': 10849.8348, 'train_samples_per_second': 1.161, 'train_steps_per_second': 0.58, 'total_flos': 7.261050116662886e+16, 'train_loss': 1.3934248072370943, 'epoch': 2.0})

# Save model & Publish

In [27]:
trainer.model.save_pretrained("./llama-3-8B-mindmate")
tokenizer.save_pretrained("./llama-3-8B-mindmate")

('./llama-3-8B-mindmate/tokenizer_config.json',
 './llama-3-8B-mindmate/special_tokens_map.json',
 './llama-3-8B-mindmate/tokenizer.json')